In [10]:
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes",
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate",
    "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
    "target"]

In [13]:
import gradio as gr
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Load the saved model
loaded_model = load_model('C:/Users/PRATHAM/Downloads/NIDS.h5')
scaler = StandardScaler()
label_encoder = LabelEncoder()

def preprocess(file):
  data = pd.read_csv(file,compression='gzip', names=column_names)
  attacks_types = {
    'normal.': 'normal', 'back.': 'dos', 'buffer_overflow.': 'u2r', 'ftp_write.': 'r2l', 'guess_passwd.': 'r2l',
    'imap.': 'r2l', 'ipsweep.': 'probe', 'land.': 'dos', 'loadmodule.': 'u2r', 'multihop.': 'r2l', 'neptune.': 'dos',
    'nmap.': 'probe', 'perl.': 'u2r', 'phf.': 'r2l', 'pod.': 'dos', 'portsweep.': 'probe', 'rootkit.': 'u2r', 'satan.': 'probe',
    'smurf.': 'dos', 'spy.': 'r2l', 'teardrop.': 'dos', 'warezclient.': 'r2l', 'warezmaster.': 'r2l'}
  data['Attack Type'] = data.target.apply(lambda r:attacks_types[r])
  categorical_indices = [1, 2, 3]
  for col_index in categorical_indices:
    data.iloc[:, col_index] = label_encoder.fit_transform(data.iloc[:, col_index])
  
  data = data.drop(['target', 'Attack Type'], axis=1)
  data = data.iloc[:,:].values
  print(data)
  return data
# Define the prediction function
def predict_csv(file):
    # Preprocess the data if needed (adjust this based on your preprocessing steps)
    data = preprocess(file)
    numerical_indices = [0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                     24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
    categorical_indices = [1, 2, 3]

    numeric = data[:, numerical_indices]
    categorical = data[:, categorical_indices]
    numeric = scaler.fit_transform(numeric)
    print(numeric.shape)
    print(categorical.shape)
    predictions = loaded_model.predict([numeric]+[data[:, index] for index in categorical_indices])
    predicted_labels = np.argmax(predictions, axis=1)

    # Creating a dictionary to map indices to categories
    index_to_category = {
        0: 'normal',
        1: 'dos',
        2: 'u2r',
        3: 'r2l',
        4: 'other'
    }

    # Mapping indices to categories using the dictionary
    predicted_categories = np.vectorize(index_to_category.get)(predicted_labels)
    return f"Network Intrusion Detected, Attack Type: {predicted_categories}"


# Defining the Gradio interface
iface = gr.Interface(
    fn=predict_csv,
    inputs="file",
    outputs="text",
    live=True,
    title="Network Intrusion Detection System",
    description="Upload a CSV file, and the model will detect whether there is any network intrusion or not and the type of attack.",
)

# Launching the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
